不動産価格予測

In [1]:
import os

In [2]:
import re

import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
sns.set(font="IPAexGothic")

from glob import glob
from functools import partial
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from xfeat import SelectCategorical, LabelEncoder, LambdaEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer

コンペデータ作成（利用カラム、目的変数など決める）

dataディレクトリ配下に以下のようにデータを格納しているとする

├─ input<br>
│   ├─ data_explanation.xlsx<br>
│   ├─ sample_submission.csv<br>
│   ├─ test.csv<br>
│   └─ train<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├─ 01.csv<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├─ 02.csv<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├─ 03.csv<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├─ ...<br>
│       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└─ 47.csv<br>
└─ output


In [3]:
paths = glob('train/*')
train_dfs = []
for path in paths:
    train_df = pd.read_csv(path)
    train_dfs.append(train_df)
train_df = pd.concat(train_dfs)
train_df.reset_index(drop=True, inplace=True)

C:\Users\zekus\AppData\Local\Temp\ipykernel_5296\492369810.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(path)
C:\Users\zekus\AppData\Local\Temp\ipykernel_5296\492369810.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(path)
C:\Users\zekus\AppData\Local\Temp\ipykernel_5296\492369810.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(path)
C:\Users\zekus\AppData\Local\Temp\ipykernel_5296\492369810.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(path)


In [4]:
test_df = pd.read_csv('test.csv')

In [5]:
sub_df = pd.read_csv('sample_submission.csv')

In [6]:
ID = 'ID'
TARGET = '取引価格（総額）_log'
rm_cols = []

In [7]:
df = pd.concat([train_df, test_df])

rm_cols += ['市区町村コード']
for i,v in df.nunique().items():
    if v <= 1:
        rm_cols.append(i)

rm_cols

['市区町村コード',
 '種類',
 '地域',
 '土地の形状',
 '間口',
 '延床面積（㎡）',
 '前面道路：方位',
 '前面道路：種類',
 '前面道路：幅員（ｍ）']

In [8]:
test_df[TARGET] = np.nan
train_df.drop(rm_cols, axis=1, inplace=True)
test_df.drop(rm_cols, axis=1, inplace=True)
df = pd.concat([train_df, test_df])
df.sort_values('取引時点', inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape

(784713, 19)

In [9]:
df['取引時点'].unique()

array(['2005年第3四半期', '2005年第4四半期', '2006年第1四半期', '2006年第2四半期',
       '2006年第3四半期', '2006年第4四半期', '2007年第1四半期', '2007年第2四半期',
       '2007年第3四半期', '2007年第4四半期', '2008年第1四半期', '2008年第2四半期',
       '2008年第3四半期', '2008年第4四半期', '2009年第1四半期', '2009年第2四半期',
       '2009年第3四半期', '2009年第4四半期', '2010年第1四半期', '2010年第2四半期',
       '2010年第3四半期', '2010年第4四半期', '2011年第1四半期', '2011年第2四半期',
       '2011年第3四半期', '2011年第4四半期', '2012年第1四半期', '2012年第2四半期',
       '2012年第3四半期', '2012年第4四半期', '2013年第1四半期', '2013年第2四半期',
       '2013年第3四半期', '2013年第4四半期', '2014年第1四半期', '2014年第2四半期',
       '2014年第3四半期', '2014年第4四半期', '2015年第1四半期', '2015年第2四半期',
       '2015年第3四半期', '2015年第4四半期', '2016年第1四半期', '2016年第2四半期',
       '2016年第3四半期', '2016年第4四半期', '2017年第1四半期', '2017年第2四半期',
       '2017年第3四半期', '2017年第4四半期', '2018年第1四半期', '2018年第2四半期',
       '2018年第3四半期', '2018年第4四半期', '2019年第1四半期', '2019年第2四半期',
       '2019年第3四半期', '2019年第4四半期', '2020年第1四半期', '2020年第2四半期',
       '2020年第3四半期', '2020年第4四半期', '2021年第1四半期', '2021年

In [10]:
val_min_idx = min(df[df['取引時点'].str.contains('2021年第4四半期|2022年第1四半期', regex=True)].index)
test_min_idx = min(df[df['取引時点'].str.contains('2022年第2四半期|2022年第3四半期', regex=True)].index)
val_min_idx, test_min_idx

(737092, 765412)

In [11]:
set(df.iloc[val_min_idx:test_min_idx, :]['取引時点'].values)

{'2021年第4四半期', '2022年第1四半期'}

特徴量生成

In [12]:
df["取引時点"].value_counts()

取引時点
2020年第4四半期    15522
2021年第2四半期    15072
2020年第1四半期    14857
2021年第1四半期    14782
2021年第4四半期    14743
              ...  
2006年第3四半期     5395
2006年第2四半期     5221
2006年第1四半期     4322
2005年第4四半期     3650
2005年第3四半期     3535
Name: count, Length: 69, dtype: int64

In [13]:
enc_dic = {}
for i, e in enumerate(sorted(list(set(df['取引時点'].values)))):
    enc_dic[e] = i
df['取引時点_enc'] = df['取引時点'].map(enc_dic)

In [14]:
te_dic = {}
time_col = '取引時点_enc'
group_col = '都道府県名'

for i in set(df[time_col].values):
    tmp_df = df[df[time_col] < i]
    te_dic[i] = tmp_df.groupby(group_col)[TARGET].agg('mean').to_dict()

te_dic[50]

{'三重県': 7.099989772836439,
 '京都府': 7.155935848555253,
 '佐賀県': 7.009937258263509,
 '兵庫県': 7.157742870483196,
 '北海道': 6.965967793677859,
 '千葉県': 7.155825324964995,
 '和歌山県': 7.022690210815773,
 '埼玉県': 7.1697442713966355,
 '大分県': 6.911537826573079,
 '大阪府': 7.158352973821201,
 '奈良県': 7.056884610234688,
 '宮城県': 7.0395442538048805,
 '宮崎県': 6.958678653711235,
 '富山県': 7.0469200713870155,
 '山口県': 7.045342730349803,
 '山形県': 7.087990602814902,
 '山梨県': 6.848839888686708,
 '岐阜県': 7.078095913486453,
 '岡山県': 7.03809073492747,
 '岩手県': 6.925002679060599,
 '島根県': 7.165376214700312,
 '広島県': 7.093449021655064,
 '徳島県': 6.912051981629745,
 '愛媛県': 6.966433011596662,
 '愛知県': 7.107301969238601,
 '新潟県': 6.913835156235044,
 '東京都': 7.377929204641064,
 '栃木県': 6.950636958071011,
 '沖縄県': 7.160046604428336,
 '滋賀県': 7.16602695816839,
 '熊本県': 6.970182251843263,
 '石川県': 6.929363348877185,
 '神奈川県': 7.2581182672098175,
 '福井県': 6.979575815840195,
 '福岡県': 6.992264905042226,
 '福島県': 6.9743805791401785,
 '秋田県': 6.9413137471126

In [15]:
tmp_df.isnull().sum()

ID                   0
都道府県名                0
市区町村名                0
地区名                672
最寄駅：名称            2796
最寄駅：距離（分）        23222
間取り              29096
面積（㎡）                0
建築年              22749
建物の構造            22207
用途               85301
今後の利用目的         368167
都市計画             20626
建ぺい率（％）          25258
容積率（％）           25258
取引時点                 0
改装               76197
取引の事情等          758791
取引価格（総額）_log     12745
取引時点_enc             0
dtype: int64

In [16]:
def calc_te(row):
    if row[time_col] in te_dic and row[group_col] in te_dic[row[time_col]]:
        return te_dic[row[time_col]][row[group_col]]
    else:
        return 0

df[group_col+'_te'] = df.apply(calc_te, axis=1)

In [17]:
def normalize_area(area):
    if area == area:
        area = int(re.sub('m\^2未満|㎡以上', '', str(area)))
    return area

In [18]:
def normalize_moyori(moyori):
    if moyori == moyori:
        if moyori == '30分?60分':
            moyori = 45
        elif moyori == '1H?1H30':
            moyori = 75
        elif moyori == '1H30?2H':
            moyori = 105
        elif moyori == '2H?':
            moyori = 120
        moyori = int(float(moyori))
    return moyori

In [19]:
def convert_wareki_to_seireki(wareki):
    if wareki == wareki:
        if wareki == '戦前':
            wareki = '昭和20年'
        value = wareki[2:-1]
        if value == '元':
            value = 1
        else:
            value = int(value)
        if '昭和' in wareki:
            seireki = 1925+value
        elif '平成' in wareki:
            seireki = 1988+value
        elif '令和' in wareki:
            seireki = 2018+value
    else:
        seireki = wareki
    return seireki

In [20]:
df['取引時点_何年前'] = df['取引時点'].apply(lambda x: 2021-int(x[:4]))
df.drop(['取引時点'], axis=1, inplace=True)
df['建築年'] = df['建築年'].apply(lambda x: convert_wareki_to_seireki(x))
df['面積（㎡）'] = df['面積（㎡）'].apply(lambda x: normalize_area(x))
df['最寄駅：距離（分）'] = df['最寄駅：距離（分）'].apply(lambda x: normalize_moyori(x))

#緯度経度取得

In [21]:
import geocoder

In [22]:
# for i,j in df["都道府県名"].value_counts().items():
#   for k,l in df[df["都道府県名"]==i]["市区町村名"].value_counts().items():
#       for m,n in df[(df["市区町村名"]==k) & (df["x"].isnull())]["地区名"].value_counts().items():
#         place=f"{i} {k} {m}"
#         ret = geocoder.osm(place, timeout=5.0) # OpenStreetMapでジオコーディング
#         ido=ret.latlng
#         print(place,ido)
#         df.loc[(df["都道府県名"]==i)&(df["市区町村名"]==k)&(df["地区名"]==k),["x","y"]]=ido
        



In [23]:
import pandas as pd
df=pd.read_csv("latlng.csv",encoding="utf-8")

C:\Users\zekus\AppData\Local\Temp\ipykernel_5296\1137521712.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("latlng.csv",encoding="utf-8")


In [12]:
stop=0
for i,j in df["都道府県名"].value_counts().items():
  for k,l in df[df["都道府県名"]==i]["市区町村名"].value_counts().items():
      for m,n in df[(df["市区町村名"]==k) & (df["x"].isnull())]["最寄駅：名称"].value_counts().items():
        place=f"{i} {k} "
        s_quote = urllib.parse.quote(place)
        makeUrl = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="
        response = requests.get(makeUrl + s_quote)
        try:
            ido=response.json()[0]["geometry"]["coordinates"][::-1]
            print(place,ido,stop)
            
            df.loc[(df["都道府県名"]==i)&(df["市区町村名"]==k)&(df["最寄駅：名称"]==m),["x","y"]]=ido
            time.sleep(0.3)
        except IndexError:
            print(place)
        



      
        

京都府 京都市中京区  [35.009998, 135.751389] 0
京都府 京都市下京区  [34.987606, 135.755478] 0
京都府 京都市伏見区  [34.936111, 135.761383] 0
京都府 京都市上京区  [35.029552, 135.756714] 0
京都府 京都市上京区  [35.029552, 135.756714] 0
京都府 京都市上京区  [35.029552, 135.756714] 0


In [9]:
df1=df[df["都道府県名"]=="京都府"]
df1[df1["x"].isnull()]

,ID,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,...,建ぺい率（％）,容積率（％）,改装,取引の事情等,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前,x,y
33,26088354,京都府,京都市右京区,太秦北路町,常盤(京都),4.0,３ＬＤＫ,75,1992.0,ＲＣ,...,60.0,200.0,未改装,NaN,7.255273,0,0.00000,16,NaN,NaN
50,26083765,京都府,京都市上京区,滋野学区,丸太町(京都市営),NaN,１Ｋ,20,1989.0,ＳＲＣ,...,80.0,600.0,未改装,NaN,6.707570,0,0.00000,16,NaN,NaN
52,26087763,京都府,京都市南区,吉祥院西ノ内町,西大路,15.0,３ＬＤＫ,90,2001.0,ＲＣ,...,60.0,200.0,改装済,NaN,7.447158,0,0.00000,16,NaN,NaN
95,26088182,京都府,京都市南区,東九条宇賀辺町,九条(京都),5.0,２ＬＤＫ,45,1993.0,ＳＲＣ,...,80.0,600.0,改装済,NaN,6.875061,0,0.00000,16,NaN,NaN
115,26083956,京都府,京都市上京区,中立学区,今出川,NaN,２ＬＤＫ,85,1989.0,ＲＣ,...,60.0,200.0,未改装,NaN,7.518514,0,0.00000,16,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784247,26017834,京都府,京都市山科区,東野門口町,東野(京都),2.0,２ＬＤＫ,50,1975.0,ＲＣ,...,80.0,600.0,改装済,NaN,NaN,68,7.18641,-1,NaN,NaN
784248,26017833,京都府,京都市山科区,東野門口町,東野(京都),3.0,２ＬＤＫ,45,1975.0,ＲＣ,...,60.0,200.0,未改装,NaN,NaN,68,7.18641,-1,NaN,NaN
784251,26017517,京都府,京都市山科区,西野左義長町,東野(京都),11.0,３ＬＤＫ,70,2016.0,ＲＣ,...,60.0,200.0,NaN,NaN,NaN,68,7.18641,-1,NaN,NaN
784257,26017331,京都府,京都市山科区,椥辻草海道町,椥辻,1.0,３ＬＤＫ,80,1998.0,ＳＲＣ,...,80.0,600.0,未改装,NaN,NaN,68,7.18641,-1,NaN,NaN


In [7]:
pre="岐阜県"
for k,l in df[df["都道府県名"]==pre]["市区町村名"].value_counts().items():
  for m,n in df[(df["市区町村名"]==k) & (df["x"].isnull())]["地区名"].value_counts().items():
    makeUrl = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="
    place=f"{pre} {k} {m}"
    s_quote = urllib.parse.quote(place)
    makeUrl = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="
    response = requests.get(makeUrl + s_quote)
    ido=response.json()[0]["geometry"]["coordinates"][::-1]
    print(place,ido)
    df.loc[(df["都道府県名"]==pre)&(df["市区町村名"]==k)&(df["地区名"]==m),["x","y"]]=ido
    time.sleep(0.5)
   



In [16]:
df.to_csv('latlng.csv', index=False)

#欠損値データの前処理

In [24]:
num_df = SelectNumerical().fit_transform(df)
num_df.head(2)

,ID,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引価格（総額）_log,取引時点_enc,都道府県名_te,取引時点_何年前,x,y
0,23223862,13.0,75,1992.0,60.0,200.0,7.30103,0,0.0,16,35.157536,136.949966
1,13324027,5.0,20,2005.0,80.0,300.0,7.39794,0,0.0,16,35.703514,139.657944


In [25]:
encoder = Pipeline([
    SelectCategorical(),
    LabelEncoder(output_suffix=""),
])

le_df = encoder.fit_transform(df)
le_df.head(30)

,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等
0,0,0,0,0,0,0,0,-1,0,0,-1
1,1,1,1,1,1,0,0,-1,1,0,-1
2,1,2,2,2,1,1,0,-1,2,0,-1
3,2,3,3,-1,2,0,0,-1,-1,0,-1
4,1,1,4,3,1,0,0,-1,1,0,-1
5,1,4,5,4,3,1,0,-1,3,1,-1
6,0,5,6,5,2,-1,0,-1,-1,0,-1
7,1,2,7,6,4,0,0,-1,4,0,-1
8,1,6,8,7,2,0,0,-1,2,0,-1
9,3,7,9,8,2,0,0,-1,0,0,-1


In [26]:
agg_dfs = []

def get_agg_df(df, group_col):

    agg_df, agg_cols = aggregation(df,
                        group_key=group_col,
                        group_values=['最寄駅：距離（分）', '面積（㎡）', '建ぺい率（％）', '容積率（％）'],
                        agg_methods=['mean'],
                        )

    return agg_df[agg_cols]

group_col_1 = '市区町村名'
group_col_2="都道府県名"
agg_dfs.append(get_agg_df(df, group_col_1))
agg_dfs.append(get_agg_df(df, group_col_2))

agg_dfs[1]

,agg_mean_最寄駅：距離（分）_grpby_都道府県名,agg_mean_面積（㎡）_grpby_都道府県名,agg_mean_建ぺい率（％）_grpby_都道府県名,agg_mean_容積率（％）_grpby_都道府県名
0,12.752583,68.076974,66.140242,275.814381
1,7.784418,47.868600,68.932924,353.752354
2,7.784418,47.868600,68.932924,353.752354
3,13.499991,58.506560,65.420474,257.171312
4,7.784418,47.868600,68.932924,353.752354
...,...,...,...,...
784708,7.784418,47.868600,68.932924,353.752354
784709,9.114751,60.217565,70.410319,335.128422
784710,9.114751,60.217565,70.410319,335.128422
784711,9.114751,60.217565,70.410319,335.128422


In [27]:
feat_df = pd.concat([num_df,le_df,agg_dfs[0],agg_dfs[1]], axis=1)
print(feat_df.shape)

(784713, 31)


In [28]:
df.isnull().sum()

ID                   0
都道府県名                0
市区町村名                0
地区名                672
最寄駅：名称            2800
最寄駅：距離（分）        23226
間取り              29520
面積（㎡）                0
建築年              22956
建物の構造            22552
用途               86921
今後の利用目的         368468
都市計画             20698
建ぺい率（％）          25357
容積率（％）           25357
改装               77297
取引の事情等          765272
取引価格（総額）_log     19301
取引時点_enc             0
都道府県名_te             0
取引時点_何年前             0
x                   14
y                   14
dtype: int64

In [29]:
feat_df.isnull().sum()

ID                                    0
最寄駅：距離（分）                         23226
面積（㎡）                                 0
建築年                               22956
建ぺい率（％）                           25357
容積率（％）                            25357
取引価格（総額）_log                      19301
取引時点_enc                              0
都道府県名_te                              0
取引時点_何年前                              0
x                                    14
y                                    14
都道府県名                                 0
市区町村名                                 0
地区名                                   0
最寄駅：名称                                0
間取り                                   0
建物の構造                                 0
用途                                    0
今後の利用目的                               0
都市計画                                  0
改装                                    0
取引の事情等                                0
agg_mean_最寄駅：距離（分）_grpby_市区町村名      256
agg_mean_面積（㎡）_grpby_市区町村名            0


In [30]:
feat_df["最寄駅：距離（分）"]=df["最寄駅：距離（分）"].fillna(feat_df["agg_mean_最寄駅：距離（分）_grpby_市区町村名"])
feat_df["最寄駅：距離（分）"]=df["最寄駅：距離（分）"].fillna(feat_df["agg_mean_最寄駅：距離（分）_grpby_都道府県名"])
feat_df["建ぺい率（％）"]=df["建ぺい率（％）"].fillna(feat_df["agg_mean_建ぺい率（％）_grpby_市区町村名"])
feat_df["建ぺい率（％）"]=df["建ぺい率（％）"].fillna(feat_df["agg_mean_建ぺい率（％）_grpby_都道府県名"])
feat_df["容積率（％）"]=df["容積率（％）"].fillna(feat_df["agg_mean_容積率（％）_grpby_市区町村名"])
feat_df["容積率（％）"]=df["容積率（％）"].fillna(feat_df["agg_mean_容積率（％）_grpby_都道府県名"])

In [31]:
encoder = Pipeline(
    [
        SelectNumerical(),
        ArithmeticCombinations(
            input_cols=["面積（㎡）", "容積率（％）"],
            drop_origin=True,
            operator="*",
            r=2,
        ),
    ]
)

num_comb_df = encoder.fit_transform(feat_df)/100
num_comb_df

,面積（㎡）容積率（％）_combi
0,150.000000
1,60.000000
2,30.000000
3,154.302787
4,45.000000
...,...
784708,180.000000
784709,120.000000
784710,120.000000
784711,260.000000


In [32]:
feat_df=pd.concat([feat_df,num_comb_df],axis=1)

In [33]:
feat_df=feat_df.drop(agg_dfs[1].columns,axis=1)


In [34]:
feat_df.isnull().sum()

ID                                    0
最寄駅：距離（分）                             0
面積（㎡）                                 0
建築年                               22956
建ぺい率（％）                               0
容積率（％）                                0
取引価格（総額）_log                      19301
取引時点_enc                              0
都道府県名_te                              0
取引時点_何年前                              0
x                                    14
y                                    14
都道府県名                                 0
市区町村名                                 0
地区名                                   0
最寄駅：名称                                0
間取り                                   0
建物の構造                                 0
用途                                    0
今後の利用目的                               0
都市計画                                  0
改装                                    0
取引の事情等                                0
agg_mean_最寄駅：距離（分）_grpby_市区町村名      256
agg_mean_面積（㎡）_grpby_市区町村名            0


モデル構築

In [35]:
train_df = feat_df.iloc[:val_min_idx, :]
val_df = feat_df.iloc[val_min_idx:test_min_idx, :]
test_df = feat_df.iloc[test_min_idx:, :]

In [36]:
feat_cols = [col for col in train_df.columns if col not in rm_cols+[ID, TARGET]]

In [37]:
cat_cols = list(le_df.columns) + ['取引時点_enc']

In [38]:
val_df.columns

Index(['ID', '最寄駅：距離（分）', '面積（㎡）', '建築年', '建ぺい率（％）', '容積率（％）', '取引価格（総額）_log',
       '取引時点_enc', '都道府県名_te', '取引時点_何年前', 'x', 'y', '都道府県名', '市区町村名', '地区名',
       '最寄駅：名称', '間取り', '建物の構造', '用途', '今後の利用目的', '都市計画', '改装', '取引の事情等',
       'agg_mean_最寄駅：距離（分）_grpby_市区町村名', 'agg_mean_面積（㎡）_grpby_市区町村名',
       'agg_mean_建ぺい率（％）_grpby_市区町村名', 'agg_mean_容積率（％）_grpby_市区町村名',
       '面積（㎡）容積率（％）_combi'],
      dtype='object')

#建築年の予測

In [39]:
feat_cols.append(TARGET)
train_x = train_df[feat_cols].dropna(subset=["建築年"]).drop("建築年",axis=1).drop(["都道府県名_te","市区町村名","地区名","都道府県名"],axis=1)
train_y = train_df["建築年"].dropna()
val_x = val_df[feat_cols].dropna(subset=["建築年"]).drop("建築年",axis=1).drop(["都道府県名_te","市区町村名","地区名","都道府県名"],axis=1)
val_y = val_df["建築年"].dropna()
test_y=feat_df[feat_df["建築年"].isnull()]["建築年"]
test_x = feat_df[feat_df["建築年"].isnull()][feat_cols].drop("建築年",axis=1).drop(["都道府県名_te","市区町村名","地区名","都道府県名"],axis=1)


In [40]:
train_x.columns

Index(['最寄駅：距離（分）', '面積（㎡）', '建ぺい率（％）', '容積率（％）', '取引時点_enc', '取引時点_何年前', 'x',
       'y', '最寄駅：名称', '間取り', '建物の構造', '用途', '今後の利用目的', '都市計画', '改装', '取引の事情等',
       'agg_mean_最寄駅：距離（分）_grpby_市区町村名', 'agg_mean_面積（㎡）_grpby_市区町村名',
       'agg_mean_建ぺい率（％）_grpby_市区町村名', 'agg_mean_容積率（％）_grpby_市区町村名',
       '面積（㎡）容積率（％）_combi', '取引価格（総額）_log'],
      dtype='object')

In [41]:
test_y.shape, test_x.shape

((22956,), (22956, 22))

In [42]:
cat_cols[3:]

['最寄駅：名称', '間取り', '建物の構造', '用途', '今後の利用目的', '都市計画', '改装', '取引の事情等', '取引時点_enc']

In [43]:
import lightgbm as lgb


In [44]:
SEED = 0

params = {
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 42,
    'max_depth': 7,
    "feature_fraction": 0.8,
    'subsample_freq': 1,
    "bagging_fraction": 0.95,
    'min_data_in_leaf': 2,
    'learning_rate': 0.1,
    "boosting": "gbdt",
    "lambda_l1": 0.1,
    "lambda_l2": 10,
    "verbosity": -1,
    "random_state": 42,
    "num_boost_round": 1000,
}

train_data = lgb.Dataset(train_x, label=train_y)
val_data = lgb.Dataset(val_x, label=val_y)

model = lgb.train(
    params,
    train_data,
    categorical_feature = cat_cols[3:],
    valid_names = ['train', 'valid'],
    valid_sets =[train_data, val_data],
    verbose_eval=100


)

val_pred = model.predict(val_x, num_iteration=model.best_iteration)
score = mean_absolute_error(val_y, val_pred)

pred_df = pd.DataFrame(sorted(zip(val_x.index, val_pred, val_y)), columns=['index', 'predict', 'actual'])

feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(), train_x.columns)), columns=['importance', 'feature'])

print(f'score: {score:.4f}')

C:\Users\zekus\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\zekus\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['今後の利用目的', '取引の事情等', '取引時点_enc', '建物の構造', '改装', '最寄駅：名称', '用途', '都市計画', '間取り']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\zekus\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\zekus\AppD

[100]	train's l1: 4.24067	valid's l1: 5.18724
[200]	train's l1: 3.87665	valid's l1: 4.92578
[300]	train's l1: 3.69151	valid's l1: 4.83284
[400]	train's l1: 3.5743	valid's l1: 4.78485
[500]	train's l1: 3.49212	valid's l1: 4.75932
[600]	train's l1: 3.43054	valid's l1: 4.75343
[700]	train's l1: 3.37722	valid's l1: 4.75263
[800]	train's l1: 3.33813	valid's l1: 4.75406
[900]	train's l1: 3.30313	valid's l1: 4.74974
[1000]	train's l1: 3.2735	valid's l1: 4.74743
score: 4.7474


In [45]:
# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(), index=train_x.columns, columns=['importance'])
display(importance.sort_values(by="importance"))

,importance
取引の事情等,79
用途,95
今後の利用目的,118
建ぺい率（％）,203
取引時点_何年前,412
改装,431
agg_mean_容積率（％）_grpby_市区町村名,458
建物の構造,498
agg_mean_建ぺい率（％）_grpby_市区町村名,541
agg_mean_最寄駅：距離（分）_grpby_市区町村名,629


In [46]:
test_pred = model.predict(test_x, num_iteration=model.best_iteration)

In [47]:
test_y=pd.Series(test_pred,index=test_y.index)

In [48]:
test_y

66        1985.397292
129       2005.039774
135       1992.249249
198       1984.370044
206       1996.264971
             ...     
784521    1994.617414
784566    1978.489226
784592    1995.708052
784680    1989.060705
784704    1996.294106
Length: 22956, dtype: float64

In [49]:
a=feat_df.copy()

In [50]:
a["建築年"]=a["建築年"].fillna(test_y)

#予測した建築年を用いた価格の推測

In [57]:
feat_cols = [col for col in train_df.columns if col not in rm_cols+[ID, TARGET]]
train_df = a.iloc[:val_min_idx, :]
val_df = a.iloc[val_min_idx:test_min_idx, :]
test_df = a.iloc[test_min_idx:, :]
train_x = train_df[feat_cols].drop(["都道府県名","市区町村名","地区名","最寄駅：名称","都道府県名_te"],axis=1)
train_y = train_df[TARGET]
val_x = val_df[feat_cols].drop(["都道府県名","市区町村名","地区名","最寄駅：名称","都道府県名_te"],axis=1)
val_y = val_df[TARGET]
test_x = test_df[feat_cols].drop(["都道府県名","市区町村名","地区名","最寄駅：名称","都道府県名_te"],axis=1)
test_y = test_df[TARGET]

In [61]:
cat_cols

['都道府県名',
 '市区町村名',
 '地区名',
 '最寄駅：名称',
 '間取り',
 '建物の構造',
 '用途',
 '今後の利用目的',
 '都市計画',
 '改装',
 '取引の事情等',
 '取引時点_enc']

In [ ]:
SEED = 0

params = {
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 6,
    'max_depth': 7,
    "feature_fraction": 0.8,
    'subsample_freq': 1,
    "bagging_fraction": 0.95,
    'min_data_in_leaf': 2,
    'learning_rate': 0.1,
    "boosting": "gbdt",
    "lambda_l1": 0.003,
    "lambda_l2": 0.0001,
    "verbosity": -1,
    "random_state": 42,
    "num_boost_round": 1000,
    'subsample_freq': 7,
    'subsample': 0.8,
    'min_child_samples': 9,
    'colsample_bytree': 1.0
}

train_data = lgb.Dataset(train_x, label=train_y)
val_data = lgb.Dataset(val_x, label=val_y)

model = lgb.train(
    params,
    train_data,
    categorical_feature = cat_cols[4:],
    valid_names = ['train', 'valid'],
    valid_sets =[train_data, val_data],
    verbose_eval=100


)

val_pred = model.predict(val_x, num_iteration=model.best_iteration)
score = mean_absolute_error(val_y, val_pred)

pred_df = pd.DataFrame(sorted(zip(val_x.index, val_pred, val_y)), columns=['index', 'predict', 'actual'])

feature_imp = pd.DataFrame(sorted(zip(model.feature_importance(), train_x.columns)), columns=['importance', 'feature'])

print(f'score: {score:.4f}')

C:\Users\zekus\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\zekus\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['今後の利用目的', '取引の事情等', '取引時点_enc', '建物の構造', '改装', '用途', '都市計画', '間取り']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\zekus\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=0.8 will be ignored. Current value: bagging_fraction=0.95
[LightGBM] [Warning] min_data_in_leaf is set=2, min_child_samples=9 will be ignored. Current value: min_data_in_leaf=2


C:\Users\zekus\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\zekus\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[100]	train's l1: 0.115379	valid's l1: 0.135156
[200]	train's l1: 0.107168	valid's l1: 0.127633
[300]	train's l1: 0.102758	valid's l1: 0.123661
[400]	train's l1: 0.0998378	valid's l1: 0.121819
[500]	train's l1: 0.0977023	valid's l1: 0.119878
[600]	train's l1: 0.0959085	valid's l1: 0.117973


In [ ]:
# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(), index=train_x.columns, columns=['importance'])
display(importance.sort_values(by="importance"))

,importance
建ぺい率（％）,11
今後の利用目的,23
都市計画,28
取引の事情等,31
取引時点_何年前,31
用途,34
建物の構造,35
容積率（％）,40
agg_mean_面積（㎡）_grpby_市区町村名,45
都道府県名,54


In [ ]:
# test_pred = model.predict(test_x, num_iteration=model.best_iteration)

In [ ]:
# test_pred

In [ ]:
# test_df[TARGET]=test_pred

In [ ]:
# test_df[TARGET]

#欠損値をゼロにする

In [ ]:
drop_col=["agg_mean_最寄駅：距離（分）_grpby_市区町村名","agg_mean_建ぺい率（％）_grpby_市区町村名","agg_mean_容積率（％）_grpby_市区町村名"]
a=a.drop(drop_col,axis=1)

In [ ]:
feat_cols = [col for col in train_df.columns if col not in rm_cols+[ID, TARGET]]
feat_cols=[col for col in feat_cols if col not in drop_col]
train_df = a.iloc[:val_min_idx, :]
val_df = a.iloc[val_min_idx:test_min_idx, :]
test_df = a.iloc[test_min_idx:, :]
train_x = train_df[feat_cols]
train_y = train_df[TARGET]
val_x = val_df[feat_cols]
val_y = val_df[TARGET]
test_x = test_df[feat_cols]
test_y = test_df[TARGET]

#いろんなモデル構築

In [ ]:
# ここで使用するライブラリ
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error as mae
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from joblib import dump, load

In [ ]:
five_mse = {}
five_r2 = {}

# 線形回帰モデルの生成
model = LinearRegression()

# モデルの学習
model.fit(train_x, train_y)

# テストデータの予測
y_pred = model.predict(val_x)

# 平均二乗誤差の計算
mae_pred = mae(val_y, y_pred)

# 平均二乗誤差の表示
print("Mean Squared Error:", mae_pred)
five_mse['lr'] = mae_pred

# モデルの当てはまり（決定係数）を確認
r2 = model.score(val_x,val_y)
print("R^2:", r2)
five_r2['lr'] = r2



Mean Squared Error: 0.16524949367901048
R^2: 0.5600018699394329


In [ ]:
# ラッソ回帰モデルの構築
lasso = Lasso(alpha=0.1)

# モデルの学習
lasso.fit(train_x, train_y)

# テストデータを用いた予測
y_pred = lasso.predict(val_x)

# 平均二乗誤差の算出
mae_pred = mae(val_y, y_pred)
print('Mean absolute Error: ', mae_pred)
five_mse['lasso'] = mae_pred

# モデルの当てはまりを確認
r2 = lasso.score(val_x, val_y)
print("R^2:", r2)
five_r2['lasso'] = r2

Mean absolute Error:  0.16929310554017868
R^2: 0.5363563143112645


In [ ]:
# リッジ回帰モデルの構築
ridge = Ridge(alpha=0.1)

# モデルの学習
ridge.fit(train_x,train_y)

# テストデータを用いた予測
y_pred = ridge.predict(val_x)

# 平均二乗誤差の算出
mae_pred = mae(val_y, y_pred)
print('Mean Squared Error: ', mae_pred)
five_mse['ridge'] = mae_pred

# モデルの当てはまりを確認
r2 = ridge.score(val_x, val_y)
print("R^2:", r2)
five_r2['ridge'] = r2

Mean Squared Error:  0.16524949233392872
R^2: 0.5600018790979677


In [ ]:
# サポートベクターリグレッションモデルの構築
# svr = SVR(kernel='linear', C=1, epsilon=0.1)

# # モデルの学習
# svr.fit(train_x, train_y)

# # テストデータを用いた予測
# y_pred = svr.predict(val_x)

# # 平均二乗誤差の算出
# mae_pred = mae(val_y, y_pred)
# print('Mean Squared Error: ', mae_pred)
# five_mse['svr'] = mae_pred

# # モデルの当てはまりを確認
# r2 = svr.score(val_x, val_y)
# print("R^2:", r2)
# five_r2['svr'] = r2

In [ ]:
# 決定木回帰モデルの構築
tree = DecisionTreeRegressor(max_depth=8, random_state=42)

# モデルの学習
tree.fit(train_x, train_y)

# テストデータを用いた予測
y_pred = tree.predict(val_x)

# 平均二乗誤差の算出
mae_pred = mae(val_y, y_pred)
print('Mean Squared Error: ', mae_pred)
five_mse['tree'] = mae_pred

# モデルの当てはまりを確認
r2 = tree.score(val_x, val_y)
print("R^2:", r2)
five_r2['tree'] = r2

Mean Squared Error:  0.1448545775021549
R^2: 0.692885036005922


In [ ]:
# maeは0に近いほどモデルの予測精度が良い
five_mse

{'lr': 0.16524949367901048,
 'lasso': 0.16929310554017868,
 'ridge': 0.16524949233392872,
 'tree': 0.1448545775021549}

#価格LightGBMでランダムサーチ

In [ ]:
train_x.columns

Index(['最寄駅：距離（分）', '面積（㎡）', '建築年', '建ぺい率（％）', '容積率（％）', '取引時点_enc',
       '都道府県名_te', '取引時点_何年前', '都道府県名', '市区町村名', '地区名', '最寄駅：名称', '間取り',
       '建物の構造', '用途', '今後の利用目的', '都市計画', '改装', '取引の事情等',
       'agg_mean_面積（㎡）_grpby_市区町村名', '面積（㎡）容積率（％）_combi'],
      dtype='object')

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
model = lgb.LGBMRegressor()

cv_params = {'reg_alpha': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1],
             'reg_lambda': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1],
             'num_leaves': [2, 3, 4, 5, 6],
             'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             'subsample': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             'subsample_freq': [0, 1, 2, 3, 4, 5, 6, 7],
             'min_child_samples': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
             }

In [ ]:
# ランダムサーチのインスタンス作成
scoring="neg_mean_absolute_error"
cv = KFold(n_splits=3, shuffle=True, random_state=123)  # KFoldでクロスバリデーション分割指定

randcv = RandomizedSearchCV(model, cv_params, cv=cv,
                            scoring=scoring, random_state=123,
                            n_iter=1120, n_jobs=-1)

randcv.fit(train_x,train_y)

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=123, shuffle=True),
                   estimator=LGBMRegressor(), n_iter=1120, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.4, 0.5, 0.6, 0.7,
                                                             0.8, 0.9, 1.0],
                                        'min_child_samples': [0, 1, 2, 3, 4, 5,
                                                              6, 7, 8, 9, 10],
                                        'num_leaves': [2, 3, 4, 5, 6],
                                        'reg_alpha': [0.0001, 0.0003, 0.001,
                                                      0.003, 0.01, 0.03, 0.1],
                                        'reg_lambda': [0.0001, 0.0003, 0.001,
                                                       0.003, 0.01, 0.03, 0.1],
                                        'subsample': [0.4, 0.5, 0.6, 0.7, 0.8,
                                                      0.9, 1.0],
                                        'subsample_freq': [0, 1, 2, 3, 4, 5, 6,
                                                           7]},
                   random_state=123, scoring='neg_mean_absolute_error')

In [ ]:
print(randcv.best_params_,randcv.best_score_)

{'subsample_freq': 7, 'subsample': 0.8, 'reg_lambda': 0.003, 'reg_alpha': 0.0001, 'num_leaves': 6, 'min_child_samples': 9, 'colsample_bytree': 1.0} -0.11413333368564944


#提出

In [ ]:
sub_df

,ID,取引価格（総額）_log
0,1000077,7.645626
1,1000081,7.629940
2,1000128,7.048015
3,1000129,6.582231
4,1000130,7.613086
...,...,...
19296,47006020,7.458711
19297,47006229,7.482280
19298,47006331,7.534564
19299,47006332,7.220994


In [ ]:
sub_df = pd.merge(sub_df[['ID']], test_df[['ID', TARGET]], on='ID')
sub_df.to_csv('test_submission.csv', index=False)